In [0]:
# ======================================================================================
# OBJETIVO: Criar um modelo de série temporal para prever o número de casos de
#           Chikungunya com base em dados históricos mensais.
# ======================================================================================

In [0]:
# --- 0. IMPORTAÇÃO DE BIBLIOTECAS ---
from pyspark.sql.functions import (
    year, month, count, expr, to_date, 
    monotonically_increasing_id, date_add, lit, trunc
)
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [0]:
# --- 1. CARREGAR, FILTRAR E AGREGAR DADOS MENSALMENTE ---
# Carrega a tabela enriquecida com dados climáticos, criada no script anterior.
df = spark.table("workspace.default.casos_completos_com_clima")

# Filtra para incluir apenas casos confirmados de Chikungunya.
df_chikungunya = df.filter(df["classificacao_final"] == "Chikungunya")

# Agrupa os casos por mês para criar a série temporal.
df_mensal = (
    df_chikungunya
      .withColumn("Mes", trunc("data_notificacao", "month")) # Usa a coluna correta 'data_notificacao'
      .groupBy("Mes")
      .agg(count("*").alias("y")) # 'y' será nossa variável alvo (número de casos)
      .orderBy("Mes")
)

# Converte o DataFrame do Spark para o Pandas para usar bibliotecas de modelagem e visualização.
pdf = df_mensal.toPandas()

In [0]:
# --- 2. ENGENHARIA DE ATRIBUTOS (FEATURES) ---
# Garante que a coluna "Mes" seja do tipo datetime.
pdf["Mes"] = pd.to_datetime(pdf["Mes"])

# Cria a variável de tendência temporal (um contador simples).
pdf["t"] = np.arange(len(pdf))

# Cria atributos de sazonalidade usando seno e cosseno para representar o ciclo anual (12 meses).
pdf["mes_do_ano"] = pdf["Mes"].dt.month.astype(float)
pdf["seno_mes"] = np.sin(2 * np.pi * pdf["mes_do_ano"] / 12)
pdf["cosseno_mes"] = np.cos(2 * np.pi * pdf["mes_do_ano"] / 12)

# Define as variáveis para o modelo (preditoras X e alvo y).
X = pdf[["t", "seno_mes", "cosseno_mes"]]
X = sm.add_constant(X) # Adiciona uma constante (intercepto) ao modelo.
y = pdf["y"]


In [0]:
# --- 3. TREINAR O MODELO ---
# Treina um modelo de Regressão Linear (Mínimos Quadrados Ordinários - OLS).
modelo = sm.OLS(y, X).fit()

In [0]:
# --- 4. PREPARAR DADOS FUTUROS PARA PREVISÃO ---

# Define quantos meses no futuro queremos prever.
meses_a_prever = 24 

# Pega a última data e o último valor de tendência dos dados históricos.
ultima_data = pdf["Mes"].max()
ultimo_t = pdf["t"].max()

# Gera as datas para os próximos meses.
datas_futuras = [ultima_data + pd.DateOffset(months=i+1) for i in range(meses_a_prever)]
pdf_futuro = pd.DataFrame(datas_futuras, columns=["Mes"])

# Cria os mesmos atributos (features) para os dados futuros.
pdf_futuro["t"] = [ultimo_t + i + 1 for i in range(meses_a_prever)]
pdf_futuro["mes_do_ano"] = pdf_futuro["Mes"].dt.month.astype(float)
pdf_futuro["seno_mes"] = np.sin(2 * np.pi * pdf_futuro["mes_do_ano"] / 12)
pdf_futuro["cosseno_mes"] = np.cos(2 * np.pi * pdf_futuro["mes_do_ano"] / 12)

# Prepara o DataFrame de atributos futuros para a previsão.
X_futuro = sm.add_constant(pdf_futuro[["t", "seno_mes", "cosseno_mes"]])

In [0]:
# --- 5. PREVER VALORES FUTUROS ---
# Usa o modelo treinado para prever os casos nos meses futuros.
pdf_futuro["casos_previstos"] = modelo.predict(X_futuro)

In [0]:
# --- 6. VISUALIZAÇÃO DOS RESULTADOS ---
print("\n--- Visualizando os resultados do modelo ---")
plt.figure(figsize=(18, 8))

# Dados históricos reais
plt.plot(pdf['Mes'], pdf['y'], label='Casos Reais Mensais', color='blue', linewidth=2)

# Ajuste do modelo sobre os dados históricos
plt.plot(pdf['Mes'], modelo.fittedvalues, label='Ajuste do Modelo (Histórico)', color='red', linestyle='--')

# Previsões futuras
plt.plot(pdf_futuro['Mes'], pdf_futuro['casos_previstos'], label=f'Previsão Futura ({meses_a_prever} meses)', color='green', marker='o', linestyle='--')

# Títulos e legendas do gráfico
plt.title('Casos Mensais de Chikungunya: Reais vs. Previstos', fontsize=16)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Número de Casos', fontsize=12)
plt.legend(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Exibe a tabela com as previsões.
print(f"\n--- PREVISÕES DE CASOS PARA OS PRÓXIMOS {meses_a_prever} MESES ---")
display(pdf_futuro[['Mes', 'casos_previstos']])

In [0]:
# --- 7. SALVAR AS PREVISÕES EM UMA TABELA ---
print("\n--- Salvando as previsões em uma tabela do Spark ---")
df_previsoes_spark = spark.createDataFrame(pdf_futuro)

nome_tabela_previsoes = "previsoes_casos_chikungunya"
df_previsoes_spark.write.mode("overwrite").saveAsTable(f"workspace.default.{nome_tabela_previsoes}")

print(f"Sucesso! As previsões foram salvas na tabela: '{nome_tabela_previsoes}'")